In [85]:
from arcgis.gis import GIS
import arcgis.network as network
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.features import Feature, FeatureSet, FeatureLayer
from arcgis.geometry import SpatialReference, project


import sys
sys.path.append('../../')
from utils import get_config

from IPython.display import HTML


In [45]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)


In [115]:
def nearest_facility(originating_address, facilities_feat_lyr, gis, as_df=False, return_all_facilities=False, all_facility_addresses=None):

    
    analysis_url = gis.properties.helperServices.closestFacility.url
    cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)
    facility_feat_list = []
    matched_facility_address = []

    if facilities_feat_lyr is not None:
        fset = facilities_feat_lyr.query()
        all_geometries = [i['geometry'] for i in fset.to_dict()['features']]
        all_attributes = [i['attributes'] for i in fset.to_dict()['features']]
        all_geometries_sr4326 = project(all_geometries ,in_sr={'latestWkid': 3857}, out_sr={'latestWkid': 4326})

        for i in range(len(all_geometries_sr4326)):
            facility_feat = Feature(geometry=all_geometries_sr4326[i],attributes=all_attributes[i])
            matched_facility_address.append(facility_feat.attributes['Match_addr'])
            facility_feat_list.append(facility_feat)
    else:      
        
        for address in all_facility_addresses:
            hit = geocode(address, max_locations=1)[0]
            facility_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])
            matched_facility_address.append(facility_feat.attributes['Match_addr'])
            facility_feat_list.append(facility_feat)

    facility_fset = FeatureSet(features=facility_feat_list, 
                                geometry_type='esriGeometryPoint', 
                                spatial_reference={'latestWkid': 4326})
    count_of_facilities = len(facility_fset)
        
    # originating_address = '581 Moss St, 91911, Chula Vista, CA'
    originating_matched_address = geocode(originating_address, max_locations=1)[0]
        
    originating_address_feature = Feature(geometry=originating_matched_address['location'], attributes=originating_matched_address['attributes'])
    originating_address_fset = FeatureSet([originating_address_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})
    
    result = cf_layer.solve_closest_facility(incidents=originating_address_fset,
                                        facilities=facility_fset,
                                        default_target_facility_count=count_of_facilities,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Miles','TravelTime'])
    line_feat_list = []
    for line_dict in result['routes']['features']:
        f1 = Feature(line_dict['geometry'], line_dict['attributes'])
        line_feat_list.append(f1)
        
    routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])
    df1 = routes_fset.sdf
    
    df1['facility_address'] = matched_facility_address
    df1['originating_address'] = [originating_address_feature.attributes['Match_addr'] for i in range(count_of_facilities)]
    df1 = df1[['facility_address','originating_address','Total_Miles','Total_TravelTime']]
    
    if return_all_facilities:
        if as_df:
            return df1
        else:
            raise ValueError('as_df should be True if return_all_facilities is True')

    
    # select the row with the minimum Total_Miles
    min_miles_index = df1['Total_Miles'].idxmin()
    min_miles_row = df1.loc[min_miles_index]
    
    return min_miles_row.to_json()


In [33]:
originating_address = '581 Moss St, 91911, Chula Vista, CA'

In [112]:
#nearest_facility(originating_address,sba_addresses, gis, return_all_facilities=False, as_df=True)

In [113]:
sba_feat_layer = FeatureLayer(gis= gis, url = "https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a8d231/FeatureServer/0")
sba_feat_layer

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a8d231/FeatureServer/0">

In [116]:
nearest_facility(originating_address, facilities_feat_lyr=sba_feat_layer, gis =gis, as_df=True, return_all_facilities=True)

,facility_address,originating_address,Total_Miles,Total_TravelTime
0,"880 National City Blvd, National City, Califor...","581 Moss St, Chula Vista, California, 91911",5.314784,8.261265
1,"880 National City Blvd, National City, Califor...","581 Moss St, Chula Vista, California, 91911",5.314784,8.261265
2,"530 B St, Ste 700, San Diego, California, 92101","581 Moss St, Chula Vista, California, 91911",5.314784,8.261265
3,"5998 Alcala Park, San Diego, California, 92110","581 Moss St, Chula Vista, California, 91911",5.314784,8.261265
4,"7675 Dagget St, Ste 340, San Diego, California...","581 Moss St, Chula Vista, California, 91911",10.671084,14.705206
5,"127 E Lexington Ave, El Cajon, California, 92020","581 Moss St, Chula Vista, California, 91911",11.376841,17.669297
6,"9605 Scranton Rd, Ste 110, San Diego, Californ...","581 Moss St, Chula Vista, California, 91911",15.552446,19.526123
7,"880 National City Blvd, National City, Califor...","581 Moss St, Chula Vista, California, 91911",16.738909,19.635598
8,"880 National City Blvd, National City, Califor...","581 Moss St, Chula Vista, California, 91911",18.400493,21.958246
9,"550 W C St, Ste 550, San Diego, California, 92101","581 Moss St, Chula Vista, California, 91911",22.271443,24.845808


In [118]:
nearest_facility(originating_address, facilities_feat_lyr=sba_feat_layer, gis =gis, as_df=False, return_all_facilities=False)


'{"facility_address":"880 National City Blvd, National City, California, 91950","originating_address":"581 Moss St, Chula Vista, California, 91911","Total_Miles":5.3147842882,"Total_TravelTime":8.2612650342}'

In [121]:
geocode("{'street':'581 Moss St','City': ','State':'CA'}")

[{'address': 'San Diego County, California',
  'location': {'x': -116.75641215299999, 'y': 33.03389816300006},
  'score': 82.21,
  'attributes': {'Loc_name': 'World',
   'Status': 'T',
   'Score': 82.21,
   'Match_addr': 'San Diego County, California',
   'LongLabel': 'San Diego County, CA, USA',
   'ShortLabel': 'San Diego County',
   'Addr_type': 'Locality',
   'Type': 'County',
   'PlaceName': 'San Diego County',
   'Place_addr': 'California',
   'Phone': '',
   'URL': '',
   'Rank': 10,
   'AddBldg': '',
   'AddNum': '',
   'AddNumFrom': '',
   'AddNumTo': '',
   'AddRange': '',
   'Side': '',
   'StPreDir': '',
   'StPreType': '',
   'StName': '',
   'StType': '',
   'StDir': '',
   'BldgType': '',
   'BldgName': '',
   'LevelType': '',
   'LevelName': '',
   'UnitType': '',
   'UnitName': '',
   'SubAddr': '',
   'StAddr': '',
   'Block': '',
   'Sector': '',
   'Nbrhd': '',
   'District': '',
   'City': '',
   'MetroArea': '',
   'Subregion': 'San Diego County',
   'Region': 'Ca